# Models

In [ ]:
from langchain.llms import LlamaCpp, OpenAI 
import config
api = config.OPENAI_API_KEY 

llm_openai = OpenAI(model_name = "text-davinci-003", openai_api_key=api)
#llm_llama = LlamaCpp(model_path="./llamacpp/models/7B/ggml-model-q4_0.bin")

In [ ]:
respuesta_openai = llm_openai("Hola, como estas?")
#respuesta_llama = llm_llama("Hola, como estas?")

In [ ]:
print(respuesta_openai)
#print(respuesta_llama)

# Chat Models

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [ ]:

chatgpt = ChatOpenAI(openai_api_key=api)

In [ ]:

respuesta = chatgpt([HumanMessage(content="Hola, como estas?")])
print(respuesta)

In [ ]:
respuesta
#respuesta.content

# Prompts

In [ ]:
from langchain import PromptTemplate

template_basico = """Eres un asistente virtual culinario que responde a preguntas
de manera muy breve.
Pregunta: Cuales son los ingredientes para preparar {platillo}
Respuesta:"""

In [ ]:
prompt_temp = PromptTemplate(input_variables=["platillo"], template = template_basico)

promt_value = prompt_temp.format(platillo="tacos al pastor")
print(promt_value)

In [ ]:
respuesta_openai = llm_openai(promt_value)
print(respuesta_openai)

In [ ]:
llm_openai.get_num_tokens(promt_value) #Install tiktoken with pip install tiktoken

# ChatPromptTemplates

In [ ]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate

In [ ]:
prompt_temp_sistema = PromptTemplate(
    template="Eres un asistente virtual que me recomienda una alternativa {adjetivo} a un producto",
    input_variables=["adjetivo"],
)

template_sistema = SystemMessagePromptTemplate(prompt=prompt_temp_sistema)

prompt_temp_humano = PromptTemplate(template="{texto}", input_variables=["texto"])

template_humano = HumanMessagePromptTemplate(prompt=prompt_temp_humano)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([template_sistema, template_humano])

chat_promt_value = chat_prompt.format_prompt(adjetivo="economica", texto="ipad").to_messages()
print(chat_promt_value)

In [ ]:
chat_resp = chatgpt(chat_promt_value)
print(chat_resp)

# Example selector

In [ ]:

from langchain import FewShotPromptTemplate

ejemplos = [
    {"pregunta": "¿Cuál es el ingrediente principal de la pizza?", "respuesta": "La masa y salsa de tomate"},
    {"pregunta": "¿Cuál es el ingrediente principal de la hamburguesa?", "respuesta": "La carne y el pan"},
    {"pregunta": "¿Cuál es el ingrediente principal del burrito?", "respuesta": "La tortilla y la carne"}
]

promt_temp_ejemplos = PromptTemplate(input_variables=["pregunta", "respuesta"], 
                                     template = "Pregunta: {pregunta}\nRespuesta: {respuesta}")

In [ ]:
prompt_ejemplos = FewShotPromptTemplate(example_prompt=promt_temp_ejemplos, 
                                       examples=ejemplos, 
                                       prefix = "Eres un asistenet virtual culinario que responde preguntas de manera muy breve",
                                       suffix = "Pregunta: {pregunta}\nRespuesta:", 
                                        input_variables=["pregunta"]) 

In [ ]:
prompt_value = promt_ejemplos.format(pregunta="¿Cuál es el ingrediente principal del coctel de camaron?")
print(prompt_value)

In [ ]:
respuesta_ingredientes = llm_openai(prompt_value)
print(respuesta_ingredientes)

In [ ]:

llm_openai("¿Cuál es el ingrediente principal del coctel de camaron?")

In [ ]:

llm_openai.get_num_tokens(prompt_value)

# Output parser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [ ]:
format_instructions

In [ ]:
template_basico_parser = """Cuales son los ingredientes para preparar {platillo}\n{como_parsear}"""

In [ ]:
prompt_temp_parser = PromptTemplate(input_variables=["platillo"], 
                                    template = template_basico_parser, 
                                    partial_variables={"como_parsear": format_instructions})

In [ ]:

promt_value_parser = prompt_temp_parser.format(platillo="tacos al pastor")

In [ ]:
print(promt_value_parser)

In [ ]:
respuesta_parser = llm_openai(promt_value_parser)

In [ ]:
respuesta_parser

In [ ]:
output_parser.parse(respuesta_parser)